In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [9]:
# Load feature-engineered data
final_df = pd.read_pickle("../../data/interim/03_final_feature_df.pkl")
final_df

,pca_1,pca_2,pca_3,participant,label,category,set,duration
epoch (ms),,,,,,,,
2019-01-11 15:08:05.200,-0.770118,-0.107077,0.590270,B,bench,heavy,30,16.8
2019-01-11 15:08:05.400,-0.753795,-0.299026,0.430993,B,bench,heavy,30,16.8
2019-01-11 15:08:05.600,-0.808422,-0.319439,0.251123,B,bench,heavy,30,16.8
2019-01-11 15:08:05.800,-0.891898,-0.159677,0.189219,B,bench,heavy,30,16.8
2019-01-11 15:08:06.000,-0.875403,-0.289579,0.328924,B,bench,heavy,30,16.8
...,...,...,...,...,...,...,...,...
2019-01-20 17:33:27.000,-0.007056,-0.101484,-0.907615,E,row,medium,90,19.4
2019-01-20 17:33:27.200,-0.000378,0.112589,-0.733069,E,row,medium,90,19.4
2019-01-20 17:33:27.400,0.002784,0.031959,-0.751999,E,row,medium,90,19.4


In [35]:
print(final_df['label'].unique())

['bench' 'ohp' 'squat' 'dead' 'row' 'rest']


In [37]:
df_model = final_df.copy()

In [39]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_model['label'] = le.fit_transform(df['label'])

In [41]:
df_model

,pca_1,pca_2,pca_3,participant,label,category,set,duration
epoch (ms),,,,,,,,
2019-01-11 15:08:05.200,-0.770118,-0.107077,0.590270,B,0,heavy,30,16.8
2019-01-11 15:08:05.400,-0.753795,-0.299026,0.430993,B,0,heavy,30,16.8
2019-01-11 15:08:05.600,-0.808422,-0.319439,0.251123,B,0,heavy,30,16.8
2019-01-11 15:08:05.800,-0.891898,-0.159677,0.189219,B,0,heavy,30,16.8
2019-01-11 15:08:06.000,-0.875403,-0.289579,0.328924,B,0,heavy,30,16.8
...,...,...,...,...,...,...,...,...
2019-01-20 17:33:27.000,-0.007056,-0.101484,-0.907615,E,4,medium,90,19.4
2019-01-20 17:33:27.200,-0.000378,0.112589,-0.733069,E,4,medium,90,19.4
2019-01-20 17:33:27.400,0.002784,0.031959,-0.751999,E,4,medium,90,19.4


In [45]:
label_mapping = dict(zip(range(len(le.classes_)), le.classes_))
print(label_mapping)


{0: 'bench', 1: 'dead', 2: 'ohp', 3: 'rest', 4: 'row', 5: 'squat'}


In [47]:
# Split data into training and test sets
# Split features and target
X = df_model[['pca_1', 'pca_2', 'pca_3']]
y = df_model['label']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [51]:
from xgboost import XGBClassifier

In [53]:
# Initialize and train XGBoost Classifier
xgb_clf = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)

In [55]:
y_pred = xgb_clf.predict(X_test)


In [57]:
# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.827968923418424
Confusion Matrix:
 [[302   0  27   0   0   4]
 [  1 248   0   1  54   2]
 [ 97   0 233   1   0   4]
 [  0   0   0 206   1  15]
 [  1  84   0   0 198   1]
 [  9   1   0   4   3 305]]
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.91      0.81       333
           1       0.74      0.81      0.78       306
           2       0.90      0.70      0.78       335
           3       0.97      0.93      0.95       222
           4       0.77      0.70      0.73       284
           5       0.92      0.95      0.93       322

    accuracy                           0.83      1802
   macro avg       0.84      0.83      0.83      1802
weighted avg       0.84      0.83      0.83      1802



In [60]:
import xgboost as xgb
model = xgb.XGBClassifier(
    objective='multi:softmax',  # or 'multi:softprob' for probabilities
    num_class=len(y.unique()),  # number of classes
    eval_metric='mlogloss',
    use_label_encoder=False,
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1
)

model.fit(X_train, y_train)


C:\Users\prajualr\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:15:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=6, ...)

In [62]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.74      0.91      0.81       333
           1       0.74      0.81      0.78       306
           2       0.90      0.70      0.78       335
           3       0.97      0.93      0.95       222
           4       0.77      0.70      0.73       284
           5       0.92      0.95      0.93       322

    accuracy                           0.83      1802
   macro avg       0.84      0.83      0.83      1802
weighted avg       0.84      0.83      0.83      1802

